In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import hddm
import os
import math
from datetime import datetime
from timeit import default_timer as timer
from datetime import timedelta
import scipy
import statistics
from zepid.graphics import EffectMeasurePlot
import pingouin as pg
from pymer4.models import Lmer

plt.close('all')
beh_path = ('/Users/nagrodzkij/data/angry/')
input_path = ('/Users/nagrodzkij/data/angry/input/')
output_path = ('/Users/nagrodzkij/data/angry/output/')

FaceScores = pd.read_csv(input_path+'/demog/FaceScores.csv',index_col=[0])

prev_depressed = pd.read_csv(input_path+'/demog/previously_depressed.csv', names=['0'])
list_depressed = prev_depressed['0'].to_list()

demog = pd.read_csv(output_path+'table_demog_byccid.csv')

################################

os.chdir(output_path+'hddm/')
hddm_path=(output_path+'hddm/')
accuracy_coding_path = hddm_path+'accuracy_coding/'
accuracy_coding_models_path=accuracy_coding_path+'models/'

data = hddm.load_csv(output_path+'data_emoFace_excl.csv')
demog = pd.read_csv(output_path+'table_demog_withedu_byccid.csv',index_col=[0])
age = demog.age
FaceScores = pd.read_csv(input_path+'/demog/FaceScores.csv',index_col=[0])
traces = pd.read_csv(accuracy_coding_models_path+'accuracy_participant1_trace.csv',index_col=0)


In [2]:
v_angry_cols = [x for x in traces.columns if 'v_subj(Angry)' in x]
v_angry_params = traces[v_angry_cols]
v_angry_params.columns = v_angry_params.columns.str[14:] #remove 't_subj' from column names

v_neutral_cols = [x for x in traces.columns if 'v_subj(Neutral)' in x]
v_neutral_params = traces[v_neutral_cols]
v_neutral_params.columns = v_neutral_params.columns.str[16:] #remove 't_subj' from column names

In [19]:
excluded_participants = ["520055", "610496"]

if set(excluded_participants).issubset(v_angry_params.columns):
    v_angry_params = v_angry_params.drop(columns=excluded_participants)

if set(excluded_participants).issubset(v_neutral_params.columns):
    v_neutral_params = v_neutral_params.drop(columns=excluded_participants)

v_angry_mean = v_angry_params.transpose()
v_angry_mean = v_angry_mean.mean(axis=1)

v_neutral_mean = v_neutral_params.transpose()
v_neutral_mean = v_neutral_mean.mean(axis=1)

stat1, p1 = scipy.stats.shapiro(v_angry_mean)
stat2, p2 = scipy.stats.shapiro(v_neutral_mean)

print(p1)
print(p2)

data_group1 = v_angry_mean
data_group2 = v_neutral_mean

if p1 and p2 > 0.05:
    print('Normally distributed, therefore performing t-test')
    tstat = pg.ttest(data_group1,data_group2,correction=True)
    print('Variance: ',np.var(data_group1), np.var(data_group2))
    print('Mean v_angry: ' + str(statistics.mean(data_group1)) + 'Mean v_neutral:' + str(statistics.mean(data_group2)))
    print('Std v_angry: '+ str(math.sqrt(np.var(data_group1))) + 'Std v_neutral: ' + str(math.sqrt(np.var(data_group2))))
    print(tstat)
    print('')

else:
    print('Not normally distributed, performing Mann Whitney U test')
    print('Median v_angry: ' + str(statistics.median(data_group1)))
    q3_depressed, q1_depressed = np.percentile(data_group1, [75 ,25])
    print('IQR v_angry: ' + str(q1_depressed) + ', ' + str(q3_depressed))

    print(' ')

    print('Median v_neutral:' + str(statistics.median(data_group2)))
    q3_nondepressed, q1_nondepressed = np.percentile(data_group2, [75 ,25])
    print('IQR v_neutral: ' + str(q1_nondepressed) + ', ' + str(q3_nondepressed))
    res = scipy.stats.mannwhitneyu(data_group1, data_group2)
    print(res)
    print('')


0.8072501420974731
0.7828725576400757
Normally distributed, therefore performing t-test
Variance:  0.3081726883250871 0.2770363500344847
Mean v_angry: 1.898998240061349Mean v_neutral:2.2188572524260604
Std v_angry: 0.5551330366003154Std v_neutral: 0.5263424265955431
               T        dof       tail     p-val           CI95%  cohen-d  \
T-test -4.840113  267.24348  two-sided  0.000002  [-0.45, -0.19]  0.58912   

            BF10   power  
T-test  6428.683  0.9979  

